In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
                X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,1192180.58,47.24,-0.66,-8.33,0.68,-0.42,0.01
4,0.0,47.24,1616104.81,31.40,5.01,7.00,-1.22,-0.04
5,0.0,-0.66,31.40,448837.04,1.97,-1.66,0.89,0.02
6,0.0,-8.33,5.01,1.97,110906.65,-0.89,-0.20,0.09
7,0.0,0.68,7.00,-1.66,-0.89,17815.93,0.08,0.00
8,0.0,-0.42,-1.22,0.89,-0.20,0.08,1525.25,-0.00
9,0.0,0.01,-0.04,0.02,0.09,0.00,-0.00,72.96


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_19717/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,0.00003,-0.00000,-0.00002,0.00000,-0.00001,0.00000
4,NaN,0.00003,1.00000,0.00004,0.00001,0.00004,-0.00002,-0.00000
5,NaN,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00003,0.00000
6,NaN,-0.00002,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00003
7,NaN,0.00000,0.00004,-0.00002,-0.00002,1.00000,0.00002,0.00000
8,NaN,-0.00001,-0.00002,0.00003,-0.00002,0.00002,1.00000,-0.00000
9,NaN,0.00000,-0.00000,0.00000,0.00003,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

82.12193398053806

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.34416547e+11 1.71149027e+11 2.38846757e+10 1.40374773e+09
 3.39756693e+07 2.21503449e+05 3.79997327e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.0,-0.212040,0.976627,-0.034308,-0.007790,-0.001241,-0.000107,-0.000005
1,-0.0,0.974403,0.208531,-0.082181,-0.017076,-0.002713,-0.000233,-0.000011
2,-0.0,0.072174,0.050166,0.994612,-0.054358,-0.008109,-0.000691,-0.000033
3,0.0,0.018802,0.013816,0.052112,0.997491,-0.041702,-0.003457,-0.000164
4,-0.0,0.003742,0.002756,0.009954,0.041022,0.998679,-0.028885,-0.001368
5,-0.0,0.000427,0.000315,0.001131,0.004586,0.028667,0.999290,-0.024002
6,-0.0,0.000031,0.000023,0.000081,0.000328,0.002048,0.023951,0.999711
7,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.025596532653325177,
 0.9498338021481125,
 0.9478879759523207,
 0.9589777006285812,
 0.9713325864899425,
 0.9760487123401212,
 1.0]